# basic intro stuff


In [4]:
%%script bash

arq --data https://raw.githubusercontent.com/Planteome/plant-trait-ontology/master/plant-trait-ontology.obo.owl '
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>  
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX oio: <http://www.geneontology.org/formats/oboInOwl#>  
PREFIX obo: <http://purl.obolibrary.org/obo/>  
        
SELECT ?object ?subject
WHERE {?subject rdfs:label ?object}
'

-----------------------------------------------------------------------------------------------------------------------------------
| object                                                             | subject                                                    |
| "transpiration rate"                                               | obo:TO_0001018                                             |
| "penultimate leaf lamina width"                                    | obo:TO_0011006                                             |
| "curved length of grain with hull"                                 | obo:TO_0002763                                             |
| "spike width"                                                      | obo:TO_0000638                                             |
| "head rice"                                                        | obo:TO_0000222                                             |
| "tassel inflorescence color"                                       | obo:T

In [8]:
%%script bash

arq --data https://raw.githubusercontent.com/Planteome/plant-trait-ontology/master/plant-trait-ontology.obo.owl '
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>  
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX oio: <http://www.geneontology.org/formats/oboInOwl#>  
PREFIX obo: <http://purl.obolibrary.org/obo/> 
        
select distinct ?properties ?name
where{?subject ?properties ?object .
      optional {?properties rdfs:label ?name}}
'        

--------------------------------------------------------
| properties              | name                       |
| owl:annotatedTarget     |                            |
| rdf:type                |                            |
| oio:hasDbXref           | "database_cross_reference" |
| owl:annotatedSource     |                            |
| owl:annotatedProperty   |                            |
| owl:intersectionOf      |                            |
| rdf:rest                |                            |
| rdf:first               |                            |
| oio:hasOBONamespace     | "has_obo_namespace"        |
| rdfs:subClassOf         |                            |
| oio:id                  |                            |
| oio:hasRelatedSynonym   | "has_related_synonym"      |
| oio:created_by          |                            |
| obo:IAO_0000115         | "definition"               |
| owl:equivalentClass     |                            |
| rdfs:label              |    

In [12]:
%%script bash

arq --data https://raw.githubusercontent.com/Planteome/plant-trait-ontology/master/plant-trait-ontology.obo.owl '
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>  
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX oio: <http://www.geneontology.org/formats/oboInOwl#>  
PREFIX obo: <http://purl.obolibrary.org/obo/> 
        
select ?object ?name
where{?object rdfs:subClassOf ?something . 
     ?something rdfs:label "plant morphology trait" . 
     ?object rdfs:label ?name }
        
'

-------------------------------------------------------
| object         | name                               |
| obo:TO_0000839 | "plant structure morphology trait" |
-------------------------------------------------------


In [17]:
%%script bash

arq --data https://raw.githubusercontent.com/Planteome/plant-trait-ontology/master/plant-trait-ontology.obo.owl '
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>  
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX oio: <http://www.geneontology.org/formats/oboInOwl#>  
PREFIX obo: <http://purl.obolibrary.org/obo/> 
        
select ?object ?name
where{?object rdfs:subClassOf obo:TO_0000017 . 
    
     ?object rdfs:label ?name }
        
'

-------------------------------------------------------
| object         | name                               |
| obo:TO_0000839 | "plant structure morphology trait" |
-------------------------------------------------------


In [19]:
%%script bash

arq --data https://raw.githubusercontent.com/Planteome/plant-trait-ontology/master/plant-trait-ontology.obo.owl '
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>  
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX oio: <http://www.geneontology.org/formats/oboInOwl#>  
PREFIX obo: <http://purl.obolibrary.org/obo/> 
        
select (count(distinct ?subject)as ?count)
where {?subject rdfs:subClassOf* obo:TO_0000017 .
      ?subject rdfs:label ?name}
        
'

---------
| count |
| 710   |
---------


In [25]:
%%script bash

arq --data https://raw.githubusercontent.com/Planteome/plant-trait-ontology/master/plant-trait-ontology.obo.owl '
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>  
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX oio: <http://www.geneontology.org/formats/oboInOwl#>  
PREFIX obo: <http://purl.obolibrary.org/obo/> 
        
select ?subject ?name
where {
    {?subject ?x ?object .
      filter(regex(str(?subject), "PATO"))}
    union 
       {?y ?x ?subject .
        ?subject rdf:type owl:Class . 
      filter(regex(str(?subject), "PATO"))}
}
      
        
'

---------------------------
| subject          | name |
| obo:PATO_0000122 |      |
| obo:PATO_0000921 |      |
| obo:PATO_0000070 |      |
| obo:PATO_0000122 |      |
| obo:PATO_0000122 |      |
| obo:PATO_0000921 |      |
| obo:PATO_0000070 |      |
| obo:PATO_0000122 |      |
| obo:PATO_0000122 |      |
| obo:PATO_0000122 |      |
| obo:PATO_0000921 |      |
| obo:PATO_0000122 |      |
| obo:PATO_0000070 |      |
| obo:PATO_0000122 |      |
| obo:PATO_0000122 |      |
| obo:PATO_0000921 |      |
| obo:PATO_0000122 |      |
| obo:PATO_0000921 |      |
| obo:PATO_0000122 |      |
| obo:PATO_0000122 |      |
| obo:PATO_0000122 |      |
| obo:PATO_0000122 |      |
| obo:PATO_0000122 |      |
| obo:PATO_0000070 |      |
| obo:PATO_0000122 |      |
| obo:PATO_0000122 |      |
| obo:PATO_0000122 |      |
| obo:PATO_0000070 |      |
| obo:PATO_0000921 |      |
| obo:PATO_0000122 |      |
| obo:PATO_0000122 |      |
| obo:PATO_0000122 |      |
| obo:PATO_0000921 |      |
| obo:PATO_0000122 |

### To which category belong a TO Term?

In [31]:
%%script bash
arq --data https://raw.githubusercontent.com/Planteome/plant-trait-ontology/master/plant-trait-ontology.obo.owl '
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>  
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX oio: <http://www.geneontology.org/formats/oboInOwl#>  
PREFIX obo: <http://purl.obolibrary.org/obo/>  
        
SELECT distinct ?to ?label ?tcLabel
WHERE { 
    ?to rdfs:subClassOf* ?topCat .
    ?topCat rdfs:subClassOf <http://purl.obolibrary.org/obo/TO_0000387> .
    ?topCat rdfs:label ?tcLabel .
    ?to rdfs:label ?label 
} 
order by ?topCat
'

------------------------------------------------------------------------------------------------------------------------------
| to             | label                                                              | tcLabel                              |
| obo:TO_0002747 | "10-dehulled grain weight"                                         | "plant morphology trait"             |
| obo:TO_0000591 | "100-dehulled grain weight"                                        | "plant morphology trait"             |
| obo:TO_0000269 | "100-seed weight"                                                  | "plant morphology trait"             |
| obo:TO_0000592 | "1000-dehulled grain weight"                                       | "plant morphology trait"             |
| obo:TO_0000382 | "1000-seed weight"                                                 | "plant morphology trait"             |
| obo:TO_0000529 | "abaxial stomatal frequency"                                       | "plant morphology trait

### TO terms with more that 1 category

In [42]:
%%script bash
arq --results TSV --data https://raw.githubusercontent.com/Planteome/plant-trait-ontology/master/plant-trait-ontology.obo.owl '
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>  
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX oio: <http://www.geneontology.org/formats/oboInOwl#>  
PREFIX obo: <http://purl.obolibrary.org/obo/>  
        
SELECT ?to ?label (group_concat(distinct ?tcLabel ; separator = " AND ") AS ?topCatTot)
WHERE { 
    ?to rdfs:subClassOf* ?topCat .
    ?topCat rdfs:subClassOf <http://purl.obolibrary.org/obo/TO_0000387> .
    ?topCat rdfs:label ?tcLabel .
    ?to rdfs:label ?label 
} 
group by ?to
having (COUNT(?topCat) > 1)
order by ?tcLabel

'

Non-group key variable in SELECT: ?label
